<a href="https://colab.research.google.com/github/massi311/horse_health_prediction/blob/main/horse_health_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/massi311/horse_health_prediction.git
%cd horse_health_prediction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)


Cloning into 'horse_health_prediction'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 65.68 KiB | 5.05 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/horse_health_prediction


In [ ]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train.loc[1,'outcome']='died'

In [ ]:
categorical_columns = ['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse',
                       'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis',
                       'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
                       'rectal_exam_feces', 'abdomen', 'abdomo_appearance',
                       'surgical_lesion', 'cp_data', 'outcome']

train= pd.get_dummies(train, columns=categorical_columns)
categorical_columns = ['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse',
                       'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis',
                       'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
                       'rectal_exam_feces', 'abdomen', 'abdomo_appearance',
                       'surgical_lesion', 'cp_data']
test=pd.get_dummies(test, columns=categorical_columns)

In [ ]:
train.head()

,id,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,...,abdomo_appearance_clear,abdomo_appearance_cloudy,abdomo_appearance_serosanguious,surgical_lesion_no,surgical_lesion_yes,cp_data_no,cp_data_yes,outcome_died,outcome_euthanized,outcome_lived
0,0,530001,38.1,132.0,24.0,6.5,57.0,8.5,3.4,2209,...,0,0,1,0,1,1,0,1,0,0
1,1,533836,37.5,88.0,12.0,2.0,33.0,64.0,2.0,2208,...,0,0,1,0,1,1,0,1,0,0
2,2,529812,38.3,120.0,28.0,3.5,37.0,6.4,3.4,5124,...,0,0,1,0,1,1,0,0,0,1
3,3,5262541,37.1,72.0,30.0,2.0,53.0,7.0,3.9,2208,...,0,1,0,0,1,0,1,0,0,1
4,4,5299629,38.0,52.0,48.0,7.0,47.0,7.3,2.6,0,...,0,1,0,1,0,0,1,0,0,1


In [ ]:
train=train.drop(["id","hospital_number","outcome_died","outcome_euthanized"],axis=1)


KeyError: ignored

In [ ]:
extra_columns = ['pain_slight', 'peristalsis_distend_small', 'nasogastric_reflux_slight', 'rectal_exam_feces_serosanguious']

# Drop these columns from the test set
train = train.drop(columns=extra_columns,axis=1)

In [ ]:
test=test.drop(["pain_moderate"],axis=1)

In [ ]:
test=test.drop(["id","hospital_number"],axis=1)

NameError: ignored

In [ ]:
X=train.drop(["outcome_lived"],axis=1)
Y=train["outcome_lived"]

In [ ]:
Y.shape


(1235,)

In [ ]:
#normalize the data
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(test)
test_n=norm_l(test)

In [ ]:
Y.shape

(1235,)

In [ ]:
Xn = np.tile(Xn,(1000,1))
Y= np.tile(Y,1000)

In [ ]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        tf.keras.Input(shape=(78,)),
        Dense(50, activation='sigmoid', name = 'layer1'),
        Dense(1, activation='sigmoid', name = 'layer2')
     ]
)
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xn,Y,
    epochs=10,
)

Epoch 1/10
38594/38594 [==============================] - 72s 2ms/step - loss: 0.0110
Epoch 2/10
38594/38594 [==============================] - 72s 2ms/step - loss: 7.7749e-04
Epoch 3/10
38594/38594 [==============================] - 71s 2ms/step - loss: 4.5355e-04
Epoch 4/10
38594/38594 [==============================] - 74s 2ms/step - loss: 2.4787e-04
Epoch 5/10
38594/38594 [==============================] - 71s 2ms/step - loss: 9.0846e-04
Epoch 6/10
38594/38594 [==============================] - 72s 2ms/step - loss: 4.2213e-04
Epoch 7/10
38594/38594 [==============================] - 72s 2ms/step - loss: 2.2962e-04
Epoch 8/10
38594/38594 [==============================] - 72s 2ms/step - loss: 1.3082e-04
Epoch 9/10
38594/38594 [==============================] - 72s 2ms/step - loss: 2.0601e-07
Epoch 10/10
38594/38594 [==============================] - 70s 2ms/step - loss: 3.7717e-09


AttributeError: ignored

In [ ]:
predictions = model.predict(test_n)

26/26 [==============================] - 0s 2ms/step


In [ ]:
threshold = 0.5

# Function to determine if a value is above the threshold
def is_above_threshold(value, threshold):
    if value >= threshold:
        return 'lived'
    else:
        return 'died'

# Apply the function to each probability
outcome = [is_above_threshold(p, threshold) for p in predictions]

In [ ]:
end=pd.read_csv("test.csv")
result=pd.DataFrame(
    {
        "id":end["id"],
        "outcome":outcome

    }
)

In [ ]:
from google.colab import files

result.to_csv('submission.csv', index=False)
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>